In [222]:
import pandas as pd
import numpy as np

READ DATASETS

In [223]:
#Read the suicides dataset
suicides_df = pd.read_csv("who_suicide_statistics.csv", encoding='latin1', delimiter=',') 
suicides_df.rename(columns={"age":"age_group"}, inplace=True)

# Read the second dataset with: incomes, forest area and inflation data + rename attributes
incomes_forest_inflation_df = pd.read_csv("incomes_forest_inflation.csv", encoding='latin1', delimiter=',') 
incomes_forest_inflation_df.rename(columns={"Country Name":"country"}, inplace=True)
incomes_forest_inflation_df.rename(columns={f"{i} [YR{i}]" : f"{i}" for i in range(2006,2017)}, inplace=True)

SUICIDES DATASET PREPROCESSING

In [224]:
#We are only interested in items from years 2006-2016, so we can drop another items
suicides_df = suicides_df[(suicides_df['year'] >= 2006) & (suicides_df['year'] <= 2016)]

#We want only to analize countries for which we have full data, so we drop items with missing values
countries_with_missing_values = suicides_df[suicides_df['suicides_no'].isnull()]['country'].unique()
suicides_df = suicides_df[~suicides_df['country'].isin(countries_with_missing_values)]
 
#We also want to drop all items, which have missing value in population column
countries_with_population_missing_values = suicides_df[suicides_df['population'].isnull()]['population'].unique()
suicides_df = suicides_df[~suicides_df['population'].isin(countries_with_population_missing_values)]

#We want to add additional attributes - suicides ratio in a given demographic group
suicides_df['suicides_ratio'] = (suicides_df['suicides_no'] / suicides_df['population']).apply(lambda x: round(x,6)*1000) 

INCOMES FOREST INFLATION - DATASET PREPROCESSING

In [225]:
# We can drop irrelevant columns
incomes_forest_inflation_df.drop(['Country Code', 'Series Code'], axis = 1, inplace = True)

# We want to have the same countries in both datasets
countries_from_suicide = suicides_df['country'].unique()
incomes_forest_inflation_df = incomes_forest_inflation_df[incomes_forest_inflation_df['country'].isin(countries_from_suicide)]

# We want to drop items with mismatching country name from suicide dataset
countries_from_incomes = incomes_forest_inflation_df['country'].unique()
suicides_df = suicides_df[suicides_df['country'].isin(countries_from_incomes)]

#We want to transform dataset to have new attributes: year and value, instead of 10 columns for years, where values were placed
incomes_forest_inflation_df = pd.melt(incomes_forest_inflation_df, id_vars = ['country', 'Series Name'], var_name = 'year', value_name = 'value')

#We want to split "Series Name" attribute into "forest_area", "adjusted_nni" and "inflation"
forest_area_df = incomes_forest_inflation_df[incomes_forest_inflation_df['Series Name'] == r"Forest area (% of land area)"]
adjusted_nni_df = incomes_forest_inflation_df[incomes_forest_inflation_df['Series Name'] == r"Adjusted net national income (current US$)"]
inflation_df = incomes_forest_inflation_df[incomes_forest_inflation_df['Series Name'] == r"Inflation, consumer prices (annual %)"]

#Now we can drop attribute "Series Name"
forest_area_df.drop("Series Name", axis='columns', inplace=True)
adjusted_nni_df.drop("Series Name", axis='columns', inplace=True)
inflation_df.drop("Series Name", axis='columns', inplace=True)

#We have to change missing values representation from ".." for NaN and drop items, which have missing values for more than 3 years
forest_area_df['value'] = forest_area_df['value'].replace("..", np.nan)
adjusted_nni_df['value'] = adjusted_nni_df['value'].replace("..", np.nan)
inflation_df['value'] =  inflation_df['value'].replace("..", np.nan)

C:\Users\user\AppData\Local\Temp\ipykernel_26372\2470035191.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forest_area_df.drop("Series Name", axis='columns', inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_26372\2470035191.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adjusted_nni_df.drop("Series Name", axis='columns', inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_26372\2470035191.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [226]:
forest_area_df.rename(columns={"value":"forest_area"}, inplace=True)
adjusted_nni_df.rename(columns={"value":"adjusted_nni"}, inplace=True)
inflation_df.rename(columns={"value":"inflation"}, inplace=True)

second_df = forest_area_df.merge(adjusted_nni_df, on=['country', 'year']).merge(inflation_df, on=['country', 'year'])
second_df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_26372\3165664308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forest_area_df.rename(columns={"value":"forest_area"}, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_26372\3165664308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adjusted_nni_df.rename(columns={"value":"adjusted_nni"}, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_26372\3165664308.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,country,year,forest_area,adjusted_nni,inflation
0,Antigua and Barbuda,2006,20.5636363636364,NaN,1.78778537419119
1,Argentina,2006,11.5028008287384,191369799309.218,NaN
2,Armenia,2006,11.6400421496312,5853473811.88319,2.89235662459002
3,Aruba,2006,2.33333333333333,2063759858.73229,3.60802438784541
4,Australia,2006,16.9810213087226,576695257963.615,3.55528773727028


In [227]:
missing_values_count = second_df.groupby('country')['forest_area'].apply(lambda x: x.isna().sum())
countries_to_remove = missing_values_count[missing_values_count > 3].index
second_df = second_df[~second_df['country'].isin(countries_to_remove)]

missing_values_count = second_df.groupby('country')['adjusted_nni'].apply(lambda x: x.isna().sum())
countries_to_remove = missing_values_count[missing_values_count > 3].index
second_df = second_df[~second_df['country'].isin(countries_to_remove)]

missing_values_count = second_df.groupby('country')['inflation'].apply(lambda x: x.isna().sum())
countries_to_remove = missing_values_count[missing_values_count > 3].index
second_df = second_df[~second_df['country'].isin(countries_to_remove)]

# To ensure that after data preprocessing both datasets have the same country set
countries_from_second_df = second_df['country'].unique()
suicides_df = suicides_df[suicides_df['country'].isin(countries_from_second_df)]

second_df['adjusted_nni'] = (second_df['adjusted_nni'].astype(float) / 1000000).round().astype(int)
second_df['forest_area'] = second_df['forest_area'].astype(float).round(2)
second_df['inflation'] = second_df['inflation'].astype(float).round(2)

#Now we can save our datasets
suicides_df.to_csv('suicides_ready_df.csv', index=False, header=True)
second_df.to_csv('forest_area_adjusted_nni_inflation_ready_df.csv', index=False, header=True)


In [228]:
suicides_df.head()

,country,year,sex,age_group,suicides_no,population,suicides_ratio
1788,Armenia,2006,female,15-24 years,0.0,290966.0,0.000
1789,Armenia,2006,female,25-34 years,2.0,215884.0,0.009
1790,Armenia,2006,female,35-54 years,5.0,450477.0,0.011
1791,Armenia,2006,female,5-14 years,0.0,203195.0,0.000
1792,Armenia,2006,female,55-74 years,8.0,238745.0,0.034
